In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from ISP import *

# To make plot interactable 
%matplotlib qt5

%load_ext autoreload
%autoreload 2


## Load RAW

In [ ]:
# img,H_raw = read_raw_simple("./data/0.ARW")

file_path = "./data/0.ARW"
# 如果用 with rawpy.imread(file_path) as H_raw: 的写法 出了with block 会报错
H_raw=rawpy.imread(file_path)
# 读取raw data
raw=H_raw.raw_image
print(f"RAW Shape:{raw.shape}")

# Information
OB=H_raw.black_level_per_channel
camera_white_level=H_raw.camera_white_level_per_channel
white_level = H_raw.white_level
print(f"OB {OB} \nWhite {white_level} \nCamera white {camera_white_level} \nMax pixel {raw.max()}") #some pixel may be larger than white_level
OB=OB[0]

wb = H_raw.camera_whitebalance
print(f"Camera wb {wb}")

H_raw.color_matrix,\
H_raw.color_desc

In [ ]:
# Plot RAW In Gray
plt.figure()
plt.figure("RAW")
plt.imshow(raw, cmap='gray')  # 使用灰度色彩映射绘制原始 RAW 数据
plt.axis('off')  # 关闭坐标轴
plt.show()


In [ ]:
#Plot RAW in color Channal
vis_raw = (raw-OB).astype(float)/(white_level-OB)

zeros = np.zeros_like(vis_raw)
rgb_raw = np.dstack([zeros,zeros,zeros])
# rgb_raw.shape

# 0,1,2 R G B
# R channel 
rgb_raw[0::2,0::2,0] = vis_raw[0::2,0::2]
# G1 G2 channal
rgb_raw[0::2,1::2,1] = vis_raw[0::2,1::2]
rgb_raw[1::2,0::2,1] = vis_raw[1::2,0::2]
# B channal
rgb_raw[1::2,1::2,2] = vis_raw[1::2,1::2]

# Plot RAW In Gray
plt.figure()
plt.figure("Colored RAW")
plt.imshow(rgb_raw)  # 使用灰度色彩映射绘制原始 RAW 数据
# plt.axis('off')  # 关闭坐标轴
plt.show()

# # 将 vis_raw 数组保存为图片
# rgb_raw_img = Image.fromarray(np.uint8(rgb_raw * 255))  # 将像素值转换为 0-255 范围内的整数
# rgb_raw_img.save('RGB_RAW.png')  # 保存图片 #需要用PNG格式 jpeg格式压缩后每个像素颜色就变了....

## BLC & Bayer Demosaic


In [ ]:
img=bayer_demosaic(raw)
img=(img-OB).astype('float32')/(white_level-OB)
img[img < 0] = 0

In [ ]:
plt.figure()
plt.title("Camera\RAW RGB")
plt.imshow(img)

In [ ]:
# Rawpy Demosaic (Not worrking)
# To achieve only demosaicing without additional processing in rawpy, you can specify the no_auto_bright parameter as True and set other processing parameters to their default values. Here's an example code snippet:
# img = H_raw.postprocess(demosaic_algorithm=rawpy.DemosaicAlgorithm.LINEAR, no_auto_bright=True,output_color=rawpy.ColorSpace.raw)


## AWB

In [ ]:
# Camera AWB
R_gain,G_gain,B_gain = H_raw.camera_whitebalance[0:3]
c_AWB_gain = np.array([R_gain/G_gain,1,B_gain/G_gain])
# c_AWB_gain = c_AWB_gain / c_AWB_gain.max()

print(f"Camera AWB RGB Gain: {c_AWB_gain}")
awb_img_c =awb(img,c_AWB_gain,normalize=False)
plt.figure()
plt.title(f"AWB {c_AWB_gain} Camera RAW")
plt.imshow(awb_img_c)

In [ ]:
# # Hand Select white balance
# # area_list = manual_wb_area(img)

# # xmin,xmax,ymin,ymax = area_list[0][0]
# # block = img[ymin:ymax,xmin:xmax,:] #[H1,W1,3]
# # rgb_vector,_=im2vector(block) #[H1*W1,3]
# # rgb_mean=rgb_vector.mean(axis=0) #[3]
# # plt.figure()
# # plt.imshow(block)

# _,rgb_mean = area_list[2]
# R_g,G_g,B_g = rgb_mean
# rgb_gain = [G_g/R_g,1,G_g/B_g]
# print(f"Maunal AWB RGB Gain: {rgb_gain}")
# awb_img =awb(img,rgb_gain,normalize=False)

# plt.figure()
# plt.title(f"Hand AWB {rgb_gain}")
# plt.imshow(awb_img)

## Gamma

In [ ]:
plt.figure()
plt.title("Correction After Gamma")
plt.imshow(gamma(awb_img_c))